In [0]:
!apt install wget

from google.colab import drive
drive.mount('/content/drive/')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
Mounted at /content/drive/


In [0]:
import cv2
import skimage.io
import skimage.transform
import pandas as pd
import numpy as np
import torch
import torchvision
import os
import sys
import time
import glob
import h5py
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data_dir = 'drive/My Drive/Datathon/Data/fb/data'
save_file = 'drive/My Drive/Datathon/Data/fb/part1d.csv'
csv_dir = 'ml_preds_csv'
png_dir = 'ml_preds_png'

files = sorted(glob.glob(os.path.join(data_dir, csv_dir,'*.csv')))
file_suffix = ['000','001','010','011','100','101','110','111','002','003','012','013','102','103','112','113','020','021','030','031','120','121','130','131','022','023','032','033','122','123','132','133','200','201','210','211','300','301','310','311','202','203','212','213','302','303','312','313','220','221','230','231','320','321','330','331','222','223','232','233','322','323','332','333']
files_sorted = []
# for suf in file_suffix:
#   for i, f in enumerate(files):
#     if(f.endswith(suf+'.csv')):
#       idx = i
#   files_sorted.append(files[idx])

# data_csv = []
# for f in files_sorted:
#   data_csv.append(pd.read_csv(f))


In [0]:
# # save a h5py file
# for i,f in enumerate(file_suffix):
#   with h5py.File(os.path.join(data_dir,'ml_preds_h5files',f+'.h5'), 'w') as hf:
#     hf.create_dataset('data',  data=data_csv[i].to_numpy())

In [0]:
# read csv
data_csv = []
for f in file_suffix:
  with h5py.File(os.path.join(data_dir,'ml_preds_h5files',f+'.h5'), 'r') as hf:
    data_csv.append(hf['data'][:])


In [0]:
coordinate_delta = 5e-6
image_top_lat = []
image_left_lon = []
image_bottom_lat = []
image_right_lon = []
for i, data in enumerate(data_csv):
  # get coordinate of leftmost pixel of each image
  px_idx = 0
  px_i, px_j = data[px_idx, 0], data[px_idx, 1]
  lat_ul, lon_ul = data[px_idx, 3], data[px_idx, 4]

  image_top_lat.append(lat_ul + px_i * coordinate_delta)
  image_left_lon.append(lon_ul - px_j * coordinate_delta)

  # get coordinate of rightmost pixel of each image
  px_idx = -1
  px_i, px_j = data[px_idx, 0], data[px_idx, 1]
  lat_br, lon_br = data[px_idx, 3], data[px_idx, 4]

  image_bottom_lat.append(lat_br - (8191 - px_i) * coordinate_delta)
  image_right_lon.append(lon_br + (8191 - px_j) * coordinate_delta)


In [0]:
# convert everything to numpy array
image_top_lat = np.asarray(image_top_lat)
image_left_lon = np.asarray(image_left_lon)
image_bottom_lat = np.asarray(image_bottom_lat)
image_right_lon = np.asarray(image_bottom_lat)

# load test facebook coordinates
fb_test_file = pd.read_csv(os.path.join('drive/My Drive/Datathon/Data/fb', 'tz_popdens_sample.csv'))

test_coordinates = np.zeros((fb_test_file.shape[0], 2))
test_coordinates[:,0] = fb_test_file['latitude'].to_numpy()
test_coordinates[:,1] = fb_test_file['longitude'].to_numpy()
# fb_test_lat = fb_test_file['latitude']
# fb_test_lon = fb_test_file['longitude']
fb_test_popden = fb_test_file['population_count']


In [0]:
lat_bins = np.zeros(8)
lon_bins = np.zeros(8)
for i in range(8):
  lat_bins[i] = np.mean(image_top_lat[i*8:(i+1)*8])
  lon_bins[i] = np.mean([image_left_lon[j] for j in np.arange(i, 64, 8)])

list_image_lat = np.digitize(test_coordinates[:,0], lat_bins, right=True) 
list_image_lon = np.digitize(test_coordinates[:,1], lon_bins, right=False)
list_image_lat[list_image_lat == 0] = 1 
list_image_lon[list_image_lon == 0] = 1
list_image_lat -= 1 # bin values are actually from 1-8
list_image_lon -= 1 
list_image_idx = list_image_lat * 8 + list_image_lon

In [0]:
def find_min_distance(pt, list_pts):
  distance = np.sqrt((pt[0] - list_pts[:,0])**2 + (pt[1] - list_pts[:,1])**2)
  idxs = distance.argmin()
  return list_pts[idxs], distance[idxs]


In [0]:
save_file = 'drive/My Drive/Datathon/Data/fb/savefiles.csv'

def lin2ind(lin_idx):
  r, c = (lin_idx // 8, lin_idx % 8)
  return r, c

def ind2lin(pt):
  idx = pt[0] * 8 + pt[1]
  return idx

list_min_dist = []
list_min_lat = []
list_min_lon = []
  
for i, data in enumerate(zip(test_coordinates, list_image_idx)):
  tc, i_image = data
  image = data_csv[i_image]
  # find location of pixel in image
  # bin_latitudes = np.arange(image_bottom_lat[i_image], image_top_lat[i_image]+coordinate_delta, coordinate_delta)
  # bin_longitudes = np.arange(image_left_lon[i_image], image_right_lon[i_image]+coordinate_delta, coordinate_delta)
  
  # r = np.digitize(tc[0], bin_latitudes)
  # c = np.digitize(tc[1], bin_longitudes)

  # find all roads
  road_lat = image[image[:,2] >= 75, 3]
  road_lon = image[image[:,2] >= 75, 4]
  
  # closest pair of points
  # pdb.set_trace()
  image_midpoint = ((image_top_lat[i_image] + image_bottom_lat[i_image]) / 2, (image_left_lon[i_image] + image_right_lon[i_image]) / 2)
  true_min_pt, true_min_dist = find_min_distance(tc, np.stack((road_lat, road_lon), axis=1))
  
  # choose neighbourhood images in the grid
  r,c = lin2ind(i_image)
  if(tc[0] < image_midpoint[0]):
    r_prime = min(r+1, 7)
    if(tc[1] < image_midpoint[1]):
      c_prime = max(c-1, 0)
    else:
      c_prime = min(c+1, 7)
  else:
    r_prime = max(r-1, 0)
    if(tc[1] < image_midpoint[1]):
      c_prime = max(c-1, 0)
    else:
      c_prime = min(c+1, 7)
    
  # find in the image if there exists a road closer in other images
  img_idxs = [(r_prime, c), (r, c_prime), (r_prime, c_prime)]
  for j in range(3):
    img_linidx = ind2lin(img_idxs[j])
    image_prime = data_csv[img_linidx]

    # find all roads
    road_lat = image_prime[image_prime[:,2] > 75, 3]
    road_lon = image_prime[image_prime[:,2] > 75, 4]
    min_pt, min_dist = find_min_distance(tc, np.stack((road_lat, road_lon), axis=1))
    if(min_dist < true_min_dist):
      true_min_dist = min_dist
      true_min_pt = min_pt

  # add to list
  with open(save_file, 'a') as f:
    f.write(''.join([str(true_min_dist),',', str(true_min_pt[0]),',', str(true_min_pt[1]),'\n']))


In [0]:
# for block proceesing - distance between points
def find_min_distance(pt, list_pts):
  num_points = pt.shape[0]
  tiled_pts = np.tile(list_pts[:, 0].T, num_points)
  distance = np.sqrt((pt[0] - tiled_pts)**2 + (pt[1] - tiled_pts)**2)
  idxs = distance.argmin(axis=1)
  return list_pts[idxs], distance[idxs]


-6.0220046303400006

In [0]:
# block process inputs to speed up processing time
def lin2ind(lin_idx):
  r, c = (lin_idx // 8, lin_idx % 8)
  return r, c

def ind2lin(pt):
  idx = pt[0] * 8 + pt[1]
  return idx

list_min_dist = []
list_min_lat = []
list_min_lon = []

pack_data = []
block_size = 1000
num_blocks = len(test_coordinates) // block_size
for i in range(num_blocks):
  pack_data.append(np.concatenate((test_coordinates[i*block_size:(i+1)*block_size], 
                                             list_image_idx[i*block_size:(i+1)*block_size][:, np.newaxis]), axis=1))
pack_data.append(np.concatenate((test_coordinates[-len(test_coordinates) % block_size:], 
                                 list_image_idx[-len(test_coordinates) % block_size:][:, np.newaxis]), axis=1))
  
for i, data in enumerate(tqdm(pack_data)):
  tc, i_image = data[:, :2], data[:, 2].astype(np.int32)
  image = [data_csv[idx][:, 2] for idx in i_image]
  all_pixels = [data_csv[idx][:, 2] for idx in i_image]
  valid_pixels = [pxs >= 75 for pxs in all_pixels]
  # find location of pixel in image
  # bin_latitudes = np.arange(image_bottom_lat[i_image], image_top_lat[i_image]+coordinate_delta, coordinate_delta)
  # bin_longitudes = np.arange(image_left_lon[i_image], image_right_lon[i_image]+coordinate_delta, coordinate_delta)
  
  # r = np.digitize(tc[0], bin_latitudes)
  # c = np.digitize(tc[1], bin_longitudes)

  # find all roads
  road_lat = image[image[:,2] >= 75, 3]
  road_lon = image[image[:,2] >= 75, 4]
  
  # closest pair of points
  # pdb.set_trace()
  image_midpoint = ((image_top_lat[i_image] + image_bottom_lat[i_image]) / 2, (image_left_lon[i_image] + image_right_lon[i_image]) / 2)
  true_min_pt, true_min_dist = find_min_distance(tc, np.stack((road_lat, road_lon), axis=1))
  
  # choose neighbourhood images in the grid
  r,c = lin2ind(i_image)
  if(tc[0] < image_midpoint[0]):
    r_prime = min(r+1, 7)
    if(tc[1] < image_midpoint[1]):
      c_prime = max(c-1, 0)
    else:
      c_prime = min(c+1, 7)
  else:
    r_prime = max(r-1, 0)
    if(tc[1] < image_midpoint[1]):
      c_prime = max(c-1, 0)
    else:
      c_prime = min(c+1, 7)
    
  # find in the image if there exists a road closer in other images
  img_idxs = [np.asarray(zip(r_prime, c)), np.asarray(zip(r, c_prime)), np.asarray(zip(r_prime, c_prime))]
  for j in range(3):
    img_linidx = ind2lin(img_idxs[j])
    image_prime = data_csv[img_linidx]

    # find all roads
    road_lat = image_prime[image_prime[:,2] >= 75, 3]
    road_lon = image_prime[image_prime[:,2] >= 75, 4]
    min_pt, min_dist = find_min_distance(tc, np.stack((road_lat, road_lon), axis=1))
    save_idxs = min_dist < true_min_dist
    true_min_dist[save_idxs] = min_dist[save_idxs]
    true_min_pt[save_idxs] = min_pt[save_idxs]
    # if(min_dist < true_min_dist):
    #   true_min_dist = min_dist
    #   true_min_pt = min_pt

  # add to list
  with open(save_file, 'a') as f:
    f.write(''.join([str(true_min_dist),',', str(true_min_pt[0]),',', str(true_min_pt[1]),'\n']))


[-5.965753671065527,
 -5.965753671065527,
 -5.965753671065527,
 -5.965753671065527,
 -5.965753671065527,
 -5.965753671065527,
 -5.965753671065527,
 -5.965753671065527,
 -6.0094592380595575,
 -6.0094592380595575,
 -6.0094592380595575,
 -6.0094592380595575,
 -6.0094592380595575,
 -6.0094592380595575,
 -6.0094592380595575,
 -6.0094592380595575,
 -6.053161295714072,
 -6.053161295714072,
 -6.053161295714072,
 -6.053161295714072,
 -6.053161295714072,
 -6.053161295714072,
 -6.053161295714072,
 -6.053161295714072,
 -6.096859818887947,
 -6.096859818887947,
 -6.096859818887947,
 -6.096859818887947,
 -6.096859818887947,
 -6.096859818887947,
 -6.096859818887947,
 -6.096859818887947,
 -6.140554782450299,
 -6.140554782450299,
 -6.140554782450299,
 -6.140554782450299,
 -6.140554782450299,
 -6.140554782450299,
 -6.140554782450299,
 -6.140554782450299,
 -6.184246161280591,
 -6.184246161280591,
 -6.184246161280591,
 -6.184246161280591,
 -6.184246161280591,
 -6.184246161280591,
 -6.184246161280591,
 -6.1